Fine-Tuning (Full Fine-Tuning) Örneği

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Önceden eğitilmiş bir modeli yüklüyoruz
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Küçük bir örnek veri seti (demo amaçlı)
dataset = load_dataset("imdb")

# Eğitim parametreleri
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=1
)

# Trainer tüm model parametrelerini güncelleyecek (Full Fine-Tuning)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"].shuffle(seed=42).select(range(1000))
)

trainer.train()


Bu örnekte modelin tüm parametreleri güncellenir. Yüksek performans sağlar ancak GPU belleği ve süre maliyeti yüksektir.

Feature Extraction Örneği

In [ ]:
for param in model.base_model.parameters():
    param.requires_grad = False  # Modelin gövdesini donduruyoruz

# Sadece classification head eğitilir
trainer.train()


Burada modelin ana katmanları sabit kalır, yalnızca son katman eğitilir.
Daha düşük maliyetlidir ancak esneklik sınırlıdır.

PEFT Tanımı – LoRA Kullanımı

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")

# LoRA konfigürasyonu
config = LoraConfig(
    r=8,                      # low-rank matris boyutu
    lora_alpha=16,
    target_modules=["c_attn"], # hangi katmanlara uygulanacağı
    lora_dropout=0.1
)

# Modeli PEFT ile sarıyoruz
peft_model = get_peft_model(model, config)

peft_model.print_trainable_parameters()


Bu kod yalnızca küçük LoRA matrislerini eğitilebilir yapar.
Ana model sabit kalır → GPU tasarrufu sağlanır.

Prefix Tuning Örneği

In [ ]:
from peft import PrefixTuningConfig, get_peft_model

config = PrefixTuningConfig(
    num_virtual_tokens=20  # modele eklenecek öğrenilebilir prefix sayısı
)

prefix_model = get_peft_model(model, config)


Model parametreleri değiştirilmez.
Sadece girişe eklenen öğrenilebilir prefix vektörleri eğitilir.

Adapter Örneği

In [ ]:
from peft import AdapterConfig

config = AdapterConfig(
    reduction_factor=16  # adapter katmanının küçültme oranı
)

adapter_model = get_peft_model(model, config)


Transformer katmanları arasına küçük adapter modülleri eklenir.
Yalnızca bu küçük modüller eğitilir.

BitFit Örneği

In [ ]:
for name, param in model.named_parameters():
    if "bias" in name:
        param.requires_grad = True   # sadece bias parametreleri eğitilir
    else:
        param.requires_grad = False


Modelin yalnızca bias terimleri güncellenir.
En düşük maliyetli fakat sınırlı adaptasyon sağlar.

RAG (Retrieval-Augmented Generation) Mini Örneği

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# Metinleri embedding'e çevirip vektör veritabanına koyuyoruz
vectorstore = FAISS.from_texts(
    ["LLM nedir?", "Transformer mimarisi self-attention kullanır."],
    OpenAIEmbeddings()
)

# Retriever oluşturuyoruz
retriever = vectorstore.as_retriever()

# LLM + Retriever zinciri
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever
)

response = qa.run("Transformer nasıl çalışır?")
print(response)


Model cevap üretmeden önce ilgili dokümanları çağırır.
Bilgi parametrelerden değil, harici kaynaktan gelir.

Haystack ile RAG Örneği

In [ ]:
from haystack.nodes import EmbeddingRetriever
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

# Dokümanları indexler
document_store.update_embeddings(retriever)


Haystack özellikle doküman arama ve QA sistemleri için optimize edilmiştir.
Endüstriyel RAG projelerinde sık kullanılır.

VQA ve Image Captioning Mini Örnek

Image Captioning

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from PIL import Image
import torch

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
processor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

image = Image.open("example.jpg")
pixel_values = processor(images=image, return_tensors="pt").pixel_values

output_ids = model.generate(pixel_values)
caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(caption)


Görsel → metin üretir.

VQA Örneği

In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

encoding = processor(image, "Bu görselde kaç kişi var?", return_tensors="pt")
outputs = model(**encoding)

logits = outputs.logits
predicted_answer = logits.argmax(-1)

print(predicted_answer)


Görsel + soru → cevap üretir.
RAG ile entegre edilirse harici bilgi de eklenebilir.